In [1]:
'''Testing for STATIC DATASET RL IS IT WORK??????'''
'''RL for Classification'''
import math
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Bernoulli
from torch.autograd import Variable
from itertools import count
import matplotlib.pyplot as plt
import random
import numpy as np
import gym
from IPython.display import clear_output



## Credit Card Data를 이용
train = pd.read_csv("./uci_creditcard-train-0.0-0.0.csv")
test = pd.read_csv("./uci_creditcard-test-0.0-0.0.csv")

train = train.to_numpy()
test = test.to_numpy()





In [2]:
'''열별로 데이터 전처리하는 과정 '''
def z_score_normalize(data) :
    normal_data = data
    for columns in range(data.shape[1]-2) :
        normal_data[:][columns] = (data[:][columns] - np.mean(data[:][columns])) /\
                                    np.std(data[:][columns])

    return normal_data


normalized_train = z_score_normalize(train)
normalized_test = z_score_normalize(test)

In [3]:
class actor_network(nn.Module):
    def __init__(self):
            super(actor_network,self).__init__()
            self.fc1 = nn.Linear(state_dim,128)
            self.fc2 = nn.Linear(128,32)
            self.fc3 = nn.Linear(32,action_dim)
            
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        ## Action의 범위 생각해야해
        return x

class critic_network(nn.Module):
    ## 근데 여기서 state
    def __init__(self,state_dim):
        super(critic_network,self).__init__()
        
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128,32)
        self.fc3 = nn.Linear(32,1)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
## Hyperparameter
eps = 0.2
gamma = 0.99
train_num = 3500
BATCH_SIZE = 100

## Input data size는 Column의 개수 -1 마지막 두개의 열은 Classification과 Valide Set을 구분.
state_dim = train.shape[1]-3
## Classification 이므로 Action은 2가 나옴.
action_dim = 2

actor = actor_network()
critic = critic_network(state_dim)
actor_optimizer = torch.optim.Adam(actor.parameters(),lr = 0.0001)
critic_optimizer = torch.optim.Adam(critic.parameters(), lr = 0.0001)



In [5]:
## Reward를 받을 때 데이터셋의 비율이 다르므로 Reward도 다르게 적용
def get_reward(action,target) :
    if action == target :
        accuracy = 1
        if action == 1 : 
            reward = 23996 / 5356
        elif action == 0 :
            reward = 23996 / 18640
    else :
        accuracy = 0
        if action == 1 :
            reward = -23996 / 5356
        elif action == 0 :
            reward = - 23996 / 18640
    
    return reward ,accuracy


In [6]:
def policy_loss(old_log_prob, log_prob, advantage, eps):
    ratio = (log_prob - old_log_prob).exp()
    clipped = torch.clamp(ratio, 1-eps, 1+eps)*advantage
    
    m = torch.min(ratio*advantage, clipped)
    return -m

In [7]:
def plot_durations(reward):
    plt.figure(2)
    plt.clf()
    reward = torch.FloatTensor(reward)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Accuracy')
    #plt.axhline(y=90,linestyle ='--')
    plt.plot(reward.numpy())
    if len(reward) >= 100:
        means = reward.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  
    
    
    
    

In [8]:
total_accuracy = []


## Traing set을 이용한 학습 
##
for i_num in range(train_num):
    ## Batch size 크기 만큼의 Trajectory가 만들어짐.
    trajectory = normalized_train[np.random.choice(normalized_train.shape[0],BATCH_SIZE,replace =False),:]
    total_acc = 0
    ## Trajectory를 만듬 길이는 BATCH_SIZE
    ## Gradient Descent

    loss = 0
    total_loss = 0
    ## Trajectory를 따라가면서 학습
    for t in range(BATCH_SIZE-1) :      
        ## state를 tensor로 바꿔서 이용
        state = trajectory[t][1:24]
        state = torch.FloatTensor(state)
        state = Variable(state)
        ## Classification Target
        target = trajectory[t][24]
        
        next_state = trajectory[t+1][1:24]
        next_state = torch.FloatTensor(state)
        next_state = Variable(next_state)
        
        
        ## Action을 선택하기 위해 이항분포 => Discrete Action space
        probs = actor(state)
        m = Bernoulli(probs)
        action = m.sample()
        prob_action = m.log_prob(action)
        action = action.data.numpy().astype(int)[0]
       
        reward,accuracy= get_reward(action,target) 
        adv = reward + gamma * critic(next_state) - critic(state)
        
        
        ## With Discout reward
        
     
        total_acc += accuracy
    
        if t > 1 :
            actor_loss = policy_loss(prev_prob_act.detach(),prob_action,adv.detach(),eps)
            actor_loss = torch.mean(actor_loss)
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            critic_loss = adv.pow(2).mean()
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            
            
        prev_prob_act = prob_action
        
    total_accuracy.append(total_acc)
    
    
    
    if (i_num+1) % 10 == 0 :
        print('Current_train_num : ',i_num, "Actor loss : ",actor_loss.item(), \
                             "Critic loss : ",critic_loss.item())
    
    if (i_num+1) % 100 == 0 :
        plot_durations(total_accuracy)
    if (i_num+1) % 500 ==0 :
        clear_output()
    
    
    
        

In [10]:
total_accuracy = []
test_num = 6004 ## Test set의 행의 크기
total_acc = 0

##Test set을 이용한 확인 
##
for i_num in range(test_num):
    
    state = normalized_test[i_num][1:24]
    
    ## state를 tensor로 바꿔서 이용
    state = torch.FloatTensor(state)
    state = Variable(state)
    ## Classification Target
    target = normalized_test[i_num][24]

    ## Action을 선택하기 위해 이항분포 => Discrete Action space
    probs = actor(state)
    m = Bernoulli(probs = probs)
    action = m.sample()
    action = action.data.numpy().astype(int)[0]
    _,accuracy= get_reward(action,target)

    total_acc += accuracy   
    total_accuracy.append(total_acc)
    if (i_num+1) % 10 == 0 :
        print('Current_test_num : ',i_num, "Accuracy : ",total_acc / i_num)

    if (i_num+1) % 100 == 0 :
        plot_durations(total_accuracy)
    if (i_num+1) % 500 ==0 :
        clear_output()
    
    

In [11]:
print("FINAL RESULT (Correct / Total Test NUM) :   ", total_acc / i_num  )
print("Total Test Num = ", test_num)

FINAL RESULT (Correct / Total Test NUM) :    0.7837747792770281
Total Test Num =  6004


Comments : PPO를 이용해서 Classification 문제를 해결하려고 시도해보았다. PG 보다 성능은 높게 나올거 같았지만 비슷했다. Reward Function에서 문제가 있는 것 같기도하다. Reward를 2배로 설정해봤지만 결과는 비슷했다.결과값을 거의 항상 0으로 뱉기때문인것 같다.